In [1]:
import pandas as pd
import numpy as np

import json
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from lightgbm import LGBMRegressor

In [2]:
train_data = pd.read_csv('train_modified.csv') # Important!!
print(train_data.shape)
train_data.head()

/home/tintash/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(903653, 56)


,Unnamed: 0,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
1,1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
2,2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,...,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
3,3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,...,NaN,NaN,NaN,(not set),NaN,NaN,google + online,organic,NaN,google
4,4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,...,NaN,NaN,NaN,(not set),NaN,True,(not provided),organic,NaN,google


In [3]:
#Print all the column names
print(train_data.columns.values)

['Unnamed: 0' 'channelGrouping' 'date' 'fullVisitorId' 'sessionId'
 'socialEngagementType' 'visitId' 'visitNumber' 'visitStartTime'
 'device.browser' 'device.browserSize' 'device.browserVersion'
 'device.deviceCategory' 'device.flashVersion' 'device.isMobile'
 'device.language' 'device.mobileDeviceBranding' 'device.mobileDeviceInfo'
 'device.mobileDeviceMarketingName' 'device.mobileDeviceModel'
 'device.mobileInputSelector' 'device.operatingSystem'
 'device.operatingSystemVersion' 'device.screenColors'
 'device.screenResolution' 'geoNetwork.city' 'geoNetwork.cityId'
 'geoNetwork.continent' 'geoNetwork.country' 'geoNetwork.latitude'
 'geoNetwork.longitude' 'geoNetwork.metro' 'geoNetwork.networkDomain'
 'geoNetwork.networkLocation' 'geoNetwork.region'
 'geoNetwork.subContinent' 'totals.bounces' 'totals.hits'
 'totals.newVisits' 'totals.pageviews' 'totals.transactionRevenue'
 'totals.visits' 'trafficSource.adContent'
 'trafficSource.adwordsClickInfo.adNetworkType'
 'trafficSource.adwordsC

In [4]:
relevantAttributes = ['channelGrouping', 'date', 'visitNumber', 
                      'device.deviceCategory', 'device.isMobile', 'device.operatingSystem', 'device.browser',
                      'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.region', 
                      'geoNetwork.subContinent',
                      'totals.hits', 'totals.pageviews', 'totals.transactionRevenue', 
                      'trafficSource.medium', 'fullVisitorId']
#                       'year', 'month', 'weekday',
#                       'mean_hits_per_day', 'mean_pageviews_per_day', 'sum_hits_per_day', 'sum_pageviews_per_day']

train_data = train_data[relevantAttributes]

In [5]:
#Figure out labeling for country param
sortedContinent = train_data.groupby('geoNetwork.continent')['totals.transactionRevenue'].mean()
sortedContinent = sortedContinent.sort_values(ascending=False)
continentCodeDict = {}
i = 0
for continent in sortedContinent.index:
    continentCodeDict[continent] = i
    i += 1
    
sortedSubContinent = train_data.groupby('geoNetwork.subContinent')['totals.transactionRevenue'].mean()
sortedSubContinent = sortedSubContinent.sort_values(ascending=False)
SubcontinentCodeDict = {}
i = 0
for subcontinent in sortedSubContinent.index:
    SubcontinentCodeDict[subcontinent] = i
    i += 1
    
sortedCountries = train_data.groupby('geoNetwork.country')['totals.transactionRevenue'].mean()
sortedCountries = sortedCountries.sort_values(ascending=False)
countryCodeDict = {}
i = 0
for country in sortedCountries.index:
    countryCodeDict[country] = i
    i += 1
    
sortedCities = train_data.groupby('geoNetwork.city')['totals.transactionRevenue'].mean()
sortedCities = sortedCities.sort_values(ascending=False)
cityCodeDict = {}
i = 0
for city in sortedCities.index:
    cityCodeDict[city] = i
    i += 1
    
sortedDevices = train_data.groupby('device.operatingSystem')['totals.transactionRevenue'].mean()
sortedDevices = sortedDevices.sort_values(ascending=False)
deviceCodeDict = {}
i = 0
for device in sortedDevices.index:
    deviceCodeDict[device] = i
    i += 1
    
sortedDevicesCategory = train_data.groupby('device.deviceCategory')['totals.transactionRevenue'].mean()
sortedDevicesCategory = sortedDevicesCategory.sort_values(ascending=False)
deviceCategoryCodeDict = {}
i = 0
for deviceCategory in sortedDevicesCategory.index:
    deviceCategoryCodeDict[deviceCategory] = i
    i += 1
    
sortedBrowser = train_data.groupby('device.browser')['totals.transactionRevenue'].mean()
sortedBrowser = sortedBrowser.sort_values(ascending=False)
browserCodeDict = {}
i = 0
for browser in sortedBrowser.index:
    browserCodeDict[browser] = i
    i += 1

sortedChannelGrouping = train_data.groupby('channelGrouping')['totals.transactionRevenue'].mean()
sortedChannelGrouping = sortedChannelGrouping.sort_values(ascending=False)
channelGroupingCodeDict = {}
i = 0
for device in sortedChannelGrouping.index:
    channelGroupingCodeDict[device] = i
    i += 1

print(channelGroupingCodeDict)
print(deviceCodeDict)
print(browserCodeDict)
print(continentCodeDict)
print(SubcontinentCodeDict)
print(deviceCategoryCodeDict)
#print(cityCodeDict)

{'Display': 0, 'Direct': 1, 'Referral': 2, 'Organic Search': 3, 'Paid Search': 4, 'Affiliates': 5, 'Social': 6, '(Other)': 7}
{'Chrome OS': 0, 'Windows': 1, 'Macintosh': 2, 'Android': 3, 'Linux': 4, 'iOS': 5, 'Windows Phone': 6, '(not set)': 7, 'BlackBerry': 8, 'Firefox OS': 9, 'FreeBSD': 10, 'NTT DoCoMo': 11, 'Nintendo 3DS': 12, 'Nintendo Wii': 13, 'Nintendo WiiU': 14, 'Nokia': 15, 'OpenBSD': 16, 'Samsung': 17, 'SunOS': 18, 'Xbox': 19}
{'Firefox': 0, 'Chrome': 1, 'Edge': 2, 'Internet Explorer': 3, 'Safari': 4, 'Opera': 5, 'Amazon Silk': 6, 'Android Webview': 7, 'Safari (in-app)': 8, '(not set)': 9, '0': 10, 'ADM': 11, 'Android Browser': 12, 'Android Runtime': 13, 'Apple-iPhone7C2': 14, 'BlackBerry': 15, 'CSM Click': 16, 'Changa 99695759': 17, 'Coc Coc': 18, 'DASH_JR_3G': 19, 'DoCoMo': 20, 'HTC802t_TD': 21, 'Hisense M20-M_LTE': 22, 'IE with Chrome Frame': 23, 'Iron': 24, 'Konqueror': 25, 'LYF_LS_4002_11': 26, 'LYF_LS_4002_12': 27, 'Lunascape': 28, 'M5': 29, 'MQQBrowser': 30, 'MRCHROME'

In [6]:
def add_time_features(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y%m%d', errors='ignore')
    df['year'] = df['date'].apply(lambda x: x.year)
    df['month'] = df['date'].apply(lambda x: x.month)
    df['day'] = df['date'].apply(lambda x: x.day)
    df['weekday'] = df['date'].apply(lambda x: x.weekday())
    
    return df

trainData = train_data.sample(frac=0.8,random_state=200)
testData = train_data.drop(trainData.index)

In [7]:
trainData['geoNetwork.subContinent'] = trainData['geoNetwork.subContinent'].transform(lambda x: SubcontinentCodeDict[x])
trainData['geoNetwork.continent'] = trainData['geoNetwork.continent'].transform(lambda x: continentCodeDict[x])
trainData['geoNetwork.country'] = trainData['geoNetwork.country'].transform(lambda x: countryCodeDict[x])
trainData['geoNetwork.city'] = trainData['geoNetwork.city'].transform(lambda x: cityCodeDict[x])
trainData['device.operatingSystem'] = trainData['device.operatingSystem'].transform(lambda x: deviceCodeDict[x])
trainData['device.browser'] = trainData['device.browser'].transform(lambda x: browserCodeDict[x])
trainData['device.deviceCategory'] = trainData['device.deviceCategory'].transform(lambda x: deviceCategoryCodeDict[x])
trainData['channelGrouping'] = trainData['channelGrouping'].transform(lambda x: channelGroupingCodeDict[x])

#Test
testData['geoNetwork.subContinent'] = testData['geoNetwork.subContinent'].transform(lambda x: SubcontinentCodeDict[x])
testData['geoNetwork.continent'] = testData['geoNetwork.continent'].transform(lambda x: continentCodeDict[x])
testData['geoNetwork.country'] = testData['geoNetwork.country'].transform(lambda x: countryCodeDict[x])
testData['geoNetwork.city'] = testData['geoNetwork.city'].transform(lambda x: cityCodeDict[x])
testData['device.operatingSystem'] = testData['device.operatingSystem'].transform(lambda x: deviceCodeDict[x])
testData['device.browser'] = testData['device.browser'].transform(lambda x: browserCodeDict[x])
testData['device.deviceCategory'] = testData['device.deviceCategory'].transform(lambda x: deviceCategoryCodeDict[x])
testData['channelGrouping'] = testData['channelGrouping'].transform(lambda x: channelGroupingCodeDict[x])

In [8]:
train = add_time_features(trainData)
test = add_time_features(testData)
# Convert target feature to 'float' type.
train["totals.transactionRevenue"] = train["totals.transactionRevenue"].astype('float')
train['totals.hits'] = train['totals.hits'].astype(float)
test['totals.hits'] = test['totals.hits'].astype(float)
train['totals.pageviews'] = train['totals.pageviews'].astype(float)
test['totals.pageviews'] = test['totals.pageviews'].astype(float)

In [9]:
gp_fullVisitorId_train = train.groupby(['fullVisitorId']).agg('sum')
gp_fullVisitorId_train['fullVisitorId'] = gp_fullVisitorId_train.index
gp_fullVisitorId_train['mean_hits_per_day'] = gp_fullVisitorId_train.groupby(['day'])['totals.hits'].transform('mean')
gp_fullVisitorId_train['mean_pageviews_per_day'] = gp_fullVisitorId_train.groupby(['day'])['totals.pageviews'].transform('mean')
gp_fullVisitorId_train['sum_hits_per_day'] = gp_fullVisitorId_train.groupby(['day'])['totals.hits'].transform('sum')
gp_fullVisitorId_train['sum_pageviews_per_day'] = gp_fullVisitorId_train.groupby(['day'])['totals.pageviews'].transform('sum')
gp_fullVisitorId_train = gp_fullVisitorId_train[['fullVisitorId', 'mean_hits_per_day', 'mean_pageviews_per_day', 'sum_hits_per_day', 'sum_pageviews_per_day']]
train = train.join(gp_fullVisitorId_train, on='fullVisitorId', how='inner', rsuffix='_')
train.drop(['fullVisitorId_'], axis=1, inplace=True)

# Test
gp_fullVisitorId_test = test.groupby(['fullVisitorId']).agg('sum')
gp_fullVisitorId_test['fullVisitorId'] = gp_fullVisitorId_test.index
gp_fullVisitorId_test['mean_hits_per_day'] = gp_fullVisitorId_test.groupby(['day'])['totals.hits'].transform('mean')
gp_fullVisitorId_test['mean_pageviews_per_day'] = gp_fullVisitorId_test.groupby(['day'])['totals.pageviews'].transform('mean')
gp_fullVisitorId_test['sum_hits_per_day'] = gp_fullVisitorId_test.groupby(['day'])['totals.hits'].transform('sum')
gp_fullVisitorId_test['sum_pageviews_per_day'] = gp_fullVisitorId_test.groupby(['day'])['totals.pageviews'].transform('sum')
gp_fullVisitorId_test = gp_fullVisitorId_test[['fullVisitorId', 'mean_hits_per_day', 'mean_pageviews_per_day', 'sum_hits_per_day', 'sum_pageviews_per_day']]
test = test.join(gp_fullVisitorId_test, on='fullVisitorId', how='inner', rsuffix='_')
test.drop(['fullVisitorId_'], axis=1, inplace=True)

In [10]:
# time_agg = train.groupby('date')['totals.transactionRevenue'].agg(['count', 'sum'])
# year_agg = train.groupby('year')['totals.transactionRevenue'].agg(['sum'])
# month_agg = train.groupby('month')['totals.transactionRevenue'].agg(['sum'])
# day_agg = train.groupby('day')['totals.transactionRevenue'].agg(['sum'])
# weekday_agg = train.groupby('weekday')['totals.transactionRevenue'].agg(['count','sum'])

In [10]:
print('TRAIN SET')
print('Rows: %s' % train.shape[0])
print('Columns: %s' % train.shape[1])
print('Features: %s' % train.columns.values)
print()
print('TEST SET')
print('Rows: %s' % test.shape[0])
print('Columns: %s' % test.shape[1])
print('Features: %s' % test.columns.values)


TRAIN SET
Rows: 722922
Columns: 25
Features: ['channelGrouping' 'date' 'visitNumber' 'device.deviceCategory'
 'device.isMobile' 'device.operatingSystem' 'device.browser'
 'geoNetwork.city' 'geoNetwork.continent' 'geoNetwork.country'
 'geoNetwork.region' 'geoNetwork.subContinent' 'totals.hits'
 'totals.pageviews' 'totals.transactionRevenue' 'trafficSource.medium'
 'fullVisitorId' 'year' 'month' 'day' 'weekday' 'mean_hits_per_day'
 'mean_pageviews_per_day' 'sum_hits_per_day' 'sum_pageviews_per_day']

TEST SET
Rows: 180731
Columns: 25
Features: ['channelGrouping' 'date' 'visitNumber' 'device.deviceCategory'
 'device.isMobile' 'device.operatingSystem' 'device.browser'
 'geoNetwork.city' 'geoNetwork.continent' 'geoNetwork.country'
 'geoNetwork.region' 'geoNetwork.subContinent' 'totals.hits'
 'totals.pageviews' 'totals.transactionRevenue' 'trafficSource.medium'
 'fullVisitorId' 'year' 'month' 'day' 'weekday' 'mean_hits_per_day'
 'mean_pageviews_per_day' 'sum_hits_per_day' 'sum_pageviews_per_

In [11]:
categorical_features = ['device.isMobile']
# , 'month', 'weekday'
train = pd.get_dummies(train, columns=categorical_features)
test = pd.get_dummies(test, columns=categorical_features)


In [12]:
# align both data sets (by outer join), to make they have the same amount of features,
# this is required because of the mismatched categorical values in train and test sets
train, test = train.align(test, join='outer', axis=1)

# replace the nan values added by align for 0
train.replace(to_replace=np.nan, value=0, inplace=True)
test.replace(to_replace=np.nan, value=0, inplace=True)

In [13]:
import datetime

X_train = train[train['date']<=datetime.date(2017, 5, 31)]
X_val = train[train['date']>datetime.date(2017, 5, 31)]
test.columns.values

/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  after removing the cwd from sys.path.


array(['channelGrouping', 'date', 'visitNumber', 'device.deviceCategory',
       'device.operatingSystem', 'device.browser', 'geoNetwork.city',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.hits', 'totals.pageviews',
       'totals.transactionRevenue', 'trafficSource.medium',
       'fullVisitorId', 'year', 'month', 'day', 'weekday',
       'mean_hits_per_day', 'mean_pageviews_per_day', 'sum_hits_per_day',
       'sum_pageviews_per_day', 'device.isMobile_False',
       'device.isMobile_True'], dtype=object)

In [14]:
# Get labels
Y_train = X_train['totals.transactionRevenue'].values
Y_val = X_val['totals.transactionRevenue'].values
X_train = X_train.drop(['totals.transactionRevenue'], axis=1)
X_val = X_val.drop(['totals.transactionRevenue'], axis=1)
Y_test = test['totals.transactionRevenue'].values
X_test = test.drop(['totals.transactionRevenue'], axis=1)
# Log transform the labels
Y_train = np.log1p(Y_train)
Y_val = np.log1p(Y_val)
Y_test = np.log1p(Y_test)


In [15]:
X_test.columns.values

array(['channelGrouping', 'date', 'visitNumber', 'device.deviceCategory',
       'device.operatingSystem', 'device.browser', 'geoNetwork.city',
       'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.hits', 'totals.pageviews',
       'trafficSource.medium', 'fullVisitorId', 'year', 'month', 'day',
       'weekday', 'mean_hits_per_day', 'mean_pageviews_per_day',
       'sum_hits_per_day', 'sum_pageviews_per_day',
       'device.isMobile_False', 'device.isMobile_True'], dtype=object)

In [16]:
reduce_features = ['year', 'month', 'day','weekday', 'date', 'fullVisitorId', 'geoNetwork.region', 
                    'trafficSource.medium']
X_train = X_train.drop(reduce_features, axis=1)
X_val = X_val.drop(reduce_features, axis=1)
X_test = X_test.drop(reduce_features, axis=1)

In [18]:
normalized_features = ['visitNumber', 'totals.hits', 'totals.pageviews', 
                       'mean_hits_per_day', 'mean_pageviews_per_day', 
                       'sum_hits_per_day', 'sum_pageviews_per_day']

# Normalize using Min-Max scaling
scaler = StandardScaler()
X_train[normalized_features] = scaler.fit_transform(X_train[normalized_features])
X_val[normalized_features] = scaler.transform(X_val[normalized_features])
test[normalized_features] = scaler.transform(test[normalized_features])

In [17]:
#Initialize LGBM
reg = LGBMRegressor(objective = 'regression', 
                     boosting_type = 'gbdt', 
                     metric = 'rmse',
                     n_estimators = 10000, #10000
                     num_leaves = 36,
                     learning_rate = 0.03, #0.01
                     bagging_fraction = 0.9,#0.8
                     feature_fraction = 0.6,#.3
                     bagging_seed = 42,
                     max_depth = 12, #-1 
#                      categorical_feature = [train_x[cat_cols].columns.get_loc(c) for c in cat_cols ] ,
                     #cat_smooth = 20
                    )

reg.fit(
    X_train, np.log1p(Y_train),
    eval_set=[(X_val, np.log1p(Y_val))],
    early_stopping_rounds=50,
    verbose=100,
    eval_metric='rmse'
)

Training until validation scores don't improve for 50 rounds.
[100]	valid_0's rmse: 0.289245
[200]	valid_0's rmse: 0.284763
[300]	valid_0's rmse: 0.284114
[400]	valid_0's rmse: 0.283868
[500]	valid_0's rmse: 0.283678
[600]	valid_0's rmse: 0.283602
Early stopping, best iteration is:
[629]	valid_0's rmse: 0.283577


LGBMRegressor(bagging_fraction=0.9, bagging_seed=42, boosting_type='gbdt',
       class_weight=None, colsample_bytree=1.0, feature_fraction=0.6,
       importance_type='split', learning_rate=0.03, max_depth=12,
       metric='rmse', min_child_samples=20, min_child_weight=0.001,
       min_split_gain=0.0, n_estimators=10000, n_jobs=-1, num_leaves=36,
       objective='regression', random_state=None, reg_alpha=0.0,
       reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)

In [18]:
#Prediction
y_pred=reg.predict(X_test , num_iteration=reg.best_iteration_)

In [19]:
testY = y_pred
y = testY

y_pred = y_pred + 1
y = y + 1

y_pred_ln = np.log1p(y_pred)
y_true_ln = np.log1p(y)

difference = np.subtract(y_pred_ln, y_true_ln)

sqr = np.square(difference)

sumSqr = sqr.sum()

sumSqrMean = sumSqr/len(y)

sumSqrRootMean = np.sqrt(sumSqrMean)
sumSqrRootMean

0.0

In [20]:
test_data = pd.read_csv('test_flat.csv', dtype={'fullVisitorId': 'str', 'visitId': 'str'}) # Important!!

test_data['pageviews'] = test_data['pageviews'].fillna(1)

X = test_data[['channelGrouping', 'date', 'visitNumber', 
                      'deviceCategory', 'isMobile', 'operatingSystem', 'browser',
                      'city', 'continent', 'country', 'region', 
                      'subContinent',
                      'hits', 'pageviews',
                      'medium', 'fullVisitorId']]

X['subContinent'] = X['subContinent'].transform(lambda x: SubcontinentCodeDict[x])
X['continent'] = X['continent'].transform(lambda x: continentCodeDict[x])
X['country'] = X['country'].transform(lambda x: countryCodeDict.get(x,220))
X['city'] = X['city'].transform(lambda x: cityCodeDict.get(x, (len(test_data['city']) -2)))
X['operatingSystem'] = X['operatingSystem'].transform(lambda x: deviceCodeDict.get(x,20))
X['browser'] = X['browser'].transform(lambda x: browserCodeDict.get(x, (len(test_data['browser'])-2)))
X['deviceCategory'] = X['deviceCategory'].transform(lambda x: deviceCategoryCodeDict[x])
X['channelGrouping'] = X['channelGrouping'].transform(lambda x: channelGroupingCodeDict[x])
test = add_time_features(X)

gp_fullVisitorId_test = test.groupby(['fullVisitorId']).agg('sum')
gp_fullVisitorId_test['fullVisitorId'] = gp_fullVisitorId_test.index
gp_fullVisitorId_test['mean_hits_per_day'] = gp_fullVisitorId_test.groupby(['day'])['hits'].transform('mean')
gp_fullVisitorId_test['mean_pageviews_per_day'] = gp_fullVisitorId_test.groupby(['day'])['pageviews'].transform('mean')
gp_fullVisitorId_test['sum_hits_per_day'] = gp_fullVisitorId_test.groupby(['day'])['hits'].transform('sum')
gp_fullVisitorId_test['sum_pageviews_per_day'] = gp_fullVisitorId_test.groupby(['day'])['pageviews'].transform('sum')
gp_fullVisitorId_test = gp_fullVisitorId_test[['fullVisitorId', 'mean_hits_per_day', 'mean_pageviews_per_day', 'sum_hits_per_day', 'sum_pageviews_per_day']]
test = test.join(gp_fullVisitorId_test, on='fullVisitorId', how='inner', rsuffix='_')
test.drop(['fullVisitorId_'], axis=1, inplace=True)

categorical_features = ['isMobile'] #, 'month', 'weekday']
test = pd.get_dummies(test, columns=categorical_features)

reduce_features = ['year', 'month', 'day','weekday', 'date', 'fullVisitorId', 'region', 
                    'medium']
test = test.drop(reduce_features, axis=1)

y_pred = reg.predict(test, num_iteration=reg.best_iteration_)

y_pred += 1

test_data['predicted'] = y_pred
# test_data[['fullVisitorId', 'predicted']].to_csv('output_row.csv')

y_pred_sum = test_data.groupby('fullVisitorId')['predicted'].sum()

y_pred_log = np.log(y_pred_sum)
        
#y_pred_log.to_csv('output_log.csv')

print(y_pred_log)

# y_pred_sum.to_csv('output_sum.csv')

/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/tintash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

fullVisitorId
0000000259678714014    0.690720
0000049363351866189    1.204781
0000053049821714864    0.068867
0000059488412965267    1.161989
0000085840370633780   -0.008193
0000091131414287111   -0.004125
0000117255350596610   -0.000537
0000118334805178127    0.001324
0000130646294093000   -0.000722
0000150005271820273    0.507260
0000166374699289385    0.373444
0000174453501096099   -0.003991
000018122977590134     0.061594
000020731284570628     0.023101
0000232022622082281   -0.001482
0000271086753662651   -0.000972
0000282648818935742   -0.000691
00003038793126460      0.764200
0000313524203455157    0.137162
0000324924635296742   -0.000197
0000338548677636278   -0.004061
0000354865008116989   -0.001307
0000359974620542953    0.695661
0000384434116640351   -0.000357
0000385653946068037   -0.003232
0000397214032106948   -0.004329
0000414003317636552    1.101026
0000421850492821864    0.018765
0000436683523507380   -0.001526
0000443488189725694    0.007635
                         .

In [21]:
submission = pd.DataFrame({'fullVisitorId':y_pred_log.index, 'PredictedLogRevenue':y_pred_log.values})
submission.to_csv("submission.csv", index=False)

In [26]:
test.columns.values

array(['channelGrouping', 'visitNumber', 'deviceCategory',
       'operatingSystem', 'browser', 'city', 'continent', 'country',
       'subContinent', 'hits', 'pageviews', 'mean_hits_per_day',
       'mean_pageviews_per_day', 'sum_hits_per_day',
       'sum_pageviews_per_day', 'isMobile_False', 'isMobile_True'],
      dtype=object)

In [30]:
test_data.shape

(804684, 54)

In [ ]:
296530